In [1]:
import sys

def is_running_in_colab():
  return 'google.colab' in sys.modules

In [2]:
%pip install --upgrade pip
%pip install wandb
%pip install randomname
%pip install --upgrade torch
%pip install flash-attn --no-build-isolation
%pip install --upgrade datasets
%pip install --upgrade accelerate
%pip install --upgrade peft
%pip install --upgrade transformers
%pip install --upgrade huggingface_hub[hf_transfer]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import randomname

name = randomname.get_name()
notes = ""

print(name)

mild-offset


In [4]:
from huggingface_hub import login
import os

if is_running_in_colab():
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
else:
    hf_token = os.environ['HF_TOKEN']
login(token=hf_token)
hf_name = "micahr234/FrozenLake-" + name

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
import wandb
import os

if is_running_in_colab():
    from google.colab import userdata
    wandb_token = userdata.get('WANDB_TOKEN')
else:
    wandb_token = os.environ['WANDB_TOKEN']
wandb.login(key=wandb_token)
wandb.init(project="FrozenLakeTrainRL", name=name, notes=notes, save_code=False)

wandb: Currently logged in as: mrentschler (mrentschler-self). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
from peft import TaskType, get_peft_model, PeftModel, LoraModel, LoraConfig, IA3Config
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import HfApi
import os

base_model_name = "meta-llama/Llama-3.1-8B-Instruct"
base_adapter_name = None #"micahr234/FrozenLake-hidden-laminate"
base_adapter_subfolder = None #"checkpoint-340"
adapter_name = None #"micahr234/FrozenLake-hidden-laminate" # only specify if you want to pickup training
adapter_subfolder = None #"checkpoint-340"
adapter_type = "ia3"

wandb.config.update({
    "base_model_name": base_model_name,
    "base_adapter_name": base_adapter_name,
    "base_adapter_subfolder": base_adapter_subfolder,
    "adapter_name": adapter_name,
    "adapter_subfolder": adapter_subfolder,
    "adapter_type": adapter_type,
})

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

hf = HfApi()
hf.snapshot_download(
    repo_id=base_model_name,
    cache_dir="./model/",
    ignore_patterns="*/*"
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    cache_dir="./model/",
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    cache_dir="./model/",
    attn_implementation="flash_attention_2",
)

if base_adapter_name is not None:
    base_model = PeftModel.from_pretrained(
        base_model,
        base_adapter_name,
        subfolder=base_adapter_subfolder + "/<default_2>",
    )
    base_model = base_model.merge_and_unload()

print(f"Memory footprint: {base_model.get_memory_footprint()/1024/1024/1024} GB")

if adapter_name is not None:
    model = PeftModel.from_pretrained(
        base_model,
        adapter_name,
        adapter_name="<default>",
        subfolder=adapter_subfolder + "/<default>",
        #mixed=True,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation="flash_attention_2",
    )

    model.load_adapter(
        adapter_name,
        "<delayed>",
        subfolder=adapter_subfolder + "/<delayed>",
        is_trainable=True,
    )

else:
    if adapter_type == "ia3":
        peft_config = IA3Config(
            #target_modules="all-linear",
            #feedforward_modules=["w0"],
            task_type=TaskType.CAUSAL_LM,
            modules_to_save=[
                #"lm_head",
                #"embed_tokens"
            ],
        )

    elif adapter_type == "lora":
        peft_config = LoraConfig(
            #target_modules="all-linear",
            task_type=TaskType.CAUSAL_LM,
            modules_to_save=[
                #"lm_head",
                #"embed_tokens"
            ],
            r=32,
            lora_alpha=32,
            lora_dropout=0.0,
            bias="none",
            use_rslora=True,
        )

    else:
        raise Exception("Unknown adapter type")

    model = get_peft_model(
        base_model,
        peft_config=peft_config,
        adapter_name="<default>",
        #mixed=True,
    )

    model.add_adapter(
        peft_config=peft_config,
        adapter_name="<delayed>",
    )

model.config.use_cache = False

model.print_trainable_parameters()

print(model)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 14.95752739906311 GB
trainable params: 524,288 || all params: 8,031,309,824 || trainable%: 0.0065
PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                (ia3_l): ParameterDict(
                    (<default>): Parameter containing: [torch.cuda.FloatTensor of size 1024x1 (cuda:0)]
                    (<delayed>): Parameter containing: [torch.cuda.BFloat16Tensor of size 1024x1 (cuda:0)]
                )
              )
              (v_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                

In [7]:
from datasets import load_dataset

dataset_name = 'micahr234/FrozenLakeNotSlipperyPrepared20eto40ecombineto4096'
percentage_of_train_data = 100
percentage_of_test_data = 100

wandb.config.update({
    "dataset_name": dataset_name,
    "percentage_of_train_data": percentage_of_train_data,
    "percentage_of_test_data": percentage_of_test_data,
})

ds_train = load_dataset(
    dataset_name,
    split=f"train[:{percentage_of_train_data}%]",
    cache_dir="./dataset/",
)
print(ds_train)

ds_test = load_dataset(
    dataset_name,
    split=f"test[:{percentage_of_test_data}%]",
    cache_dir="./dataset/",
)
print(ds_test)

README.md:   0%|          | 0.00/641 [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/4.59M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/4.55M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/4.58M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/9556 [00:00<?, ? examples/s]

Dataset({
    features: ['cumulative_reward', 'input_ids', 'attention_mask', 'action_ids_mask', 'end_ids_mask', 'env_id', 'labels'],
    num_rows: 9556
})
Dataset({
    features: ['cumulative_reward', 'input_ids', 'attention_mask', 'action_ids_mask', 'end_ids_mask', 'env_id', 'labels'],
    num_rows: 43
})


In [13]:
import torch
from transformers import Trainer, TrainerCallback
from collections import defaultdict
import inspect

action_value_discount = 0.9
reward_scale = 30.0
reward_offset = 0.0
polyak_const = 0.1
episode_value_discount = 0.0
value_weight = 1.0
evalue_weight = 0.0
imitation_weight = 0.0
observation_weight = 0.0

#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

wandb.config.update({
    "action_value_discount": action_value_discount,
    "reward_scale": reward_scale,
    "reward_offset": reward_offset,
    "polyak_const": polyak_const,
    "episode_value_discount": episode_value_discount,
    "value_weight": value_weight,
    "evalue_weight": evalue_weight,
    "imitation_weight": imitation_weight,
    "observation_weight": observation_weight,
})

def polyak_dict(source, target, tau):
    for sk, tk in zip(source.keys(), target.keys()):
        assert tk == sk
        p_source = source[tk]
        p_target = target[tk]
        p_target.data.copy_(tau * p_source.data + (1 - tau) * p_target.data)

def get_adapter_state_dict(model, adapter_name):
    state_dict = model.state_dict()
    adapter_state_dict = {k.replace(f".{adapter_name}", ".adapter_name"): state_dict[k] for k in state_dict if f".{adapter_name}" in k}
    return adapter_state_dict

def index_of_next_nonzero(x, default=-1):
    positions_of_ones = torch.nonzero(x).squeeze()
    indices = torch.arange(x.numel(), device=x.device)
    search_indices = torch.searchsorted(positions_of_ones, indices, right=False)

    y = torch.full(x.shape, device=x.device, fill_value=default, dtype=torch.int64)
    has_next_one = search_indices < positions_of_ones.numel()
    y[has_next_one] = positions_of_ones[search_indices[has_next_one]]

    return y

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._stored_metrics = defaultdict(lambda: defaultdict(list))
        self._update_count = 0

    def dqn_loss(self, outputs_1, outputs_1_old, inputs, num_items_in_batch=None):
        input_ids = inputs['input_ids'][:, 1:].detach()
        logits_1 = outputs_1['logits'][:, :-1, ...]
        logits_1_old = outputs_1_old['logits'][:, :-1, ...].detach()
        action_ids_mask = inputs['action_ids_mask'][:, 1:].detach()
        end_ids_mask = inputs['end_ids_mask'][:, 1:].detach()
        cumulative_reward = inputs['cumulative_reward'][:, 1:].detach()
        env_id = inputs['env_id'][:, 1:].detach()

        metrics_list = []
        loss_list = []
        for am, em, cr, lo, olo, ii, vd in zip(action_ids_mask, end_ids_mask, cumulative_reward, logits_1, logits_1_old, input_ids, env_id):
            with torch.no_grad():
                nm = (~am).float()
                am = am.float()
                em = em.float()
                cr = cr.float()

                episode_num = torch.cumsum(em, dim=-1) - em

                action_indices = torch.nonzero(am).squeeze(-1)
                observation_indices = torch.nonzero(nm).squeeze(-1)

                current_action_indices = action_indices[:-1]
                next_action_indices = action_indices[1:]

                current_action_episode_num = episode_num[current_action_indices]
                next_action_episode_num = episode_num[next_action_indices]

                current_episode_end = (current_action_episode_num != next_action_episode_num).float() # [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
                current_episode_reward = cr[current_action_indices]
                current_reward = reward_scale * (current_episode_reward - reward_offset) * current_episode_end

                current_action_env_id = vd[current_action_indices]
                next_action_env_id = vd[next_action_indices]

                current_env_end = (current_action_env_id != next_action_env_id).float()

                current_episode_action_i_of_i = index_of_next_nonzero(current_episode_end, default=-1) # [2, 2, 2, 8, 8, 8, 8, 8, 8, 14, 14, 14, 14, 14, 14, -1, -1]
                next_episode_action_indices = next_action_indices[current_episode_action_i_of_i] # maps back to

                current_action = ii[current_action_indices]

            next_max_value_indices = lo[next_action_indices, :].argmax(dim=-1)
            next_max_value = olo[next_action_indices, next_max_value_indices]

            target_value = current_reward + action_value_discount * (1.0 - current_episode_end) * next_max_value

            current_value = lo[current_action_indices, current_action]
            normalized_value_loss = ((current_value - target_value) / target_value.mean().detach())**2

            next_max_evalue_indices = lo[next_episode_action_indices, :].argmax(dim=-1)
            next_max_evalue = olo[next_episode_action_indices, next_max_evalue_indices]

            target_evalue = target_value.detach() + episode_value_discount * (1.0 - current_env_end) * next_max_evalue

            current_evalue = lo[current_action_indices, current_action]
            normalized_evalue_loss = ((current_evalue - target_evalue) / target_evalue.mean().detach())**2

            logprob = torch.nn.functional.log_softmax(lo, dim=-1)

            with torch.no_grad():
                action = ii[action_indices]
                observation = ii[observation_indices]

            action_logprob = logprob[action_indices, action]
            imitation_loss = -action_logprob

            observation_logprob = logprob[observation_indices, observation]
            observation_loss = -observation_logprob

            metrics_list.append({
                f'normalized_value_loss': normalized_value_loss,
                f'normalized_evalue_loss': normalized_evalue_loss,
                f'current_value': current_value,
                f'target_value': target_value,
                f'imitation_loss': imitation_loss,
                f'observation_loss': observation_loss,
            })

            loss_list.append({
                f'normalized_value_loss': normalized_value_loss * value_weight,
                f'normalized_evalue_loss': normalized_evalue_loss * evalue_weight,
                f'imitation_loss': imitation_loss * imitation_weight,
                f'observation_loss': observation_loss * observation_weight,
            })

        metrics = {key: torch.cat([i[key] for i in metrics_list], dim=-1).mean().detach().cpu() for key in metrics_list[0]}

        loss_dict = {key: torch.cat([i[key] for i in loss_list], dim=-1).mean() for key in loss_list[0]}
        loss = sum(loss_dict.values())

        return loss, metrics

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        if self.model_accepts_loss_kwargs:
            loss_kwargs = {}
            if num_items_in_batch is not None:
                loss_kwargs["num_items_in_batch"] = num_items_in_batch
            inputs = {**inputs, **loss_kwargs}

        filtered_inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}

        with torch.no_grad():
            model.set_adapter("<delayed>")
            old_outputs = model(**filtered_inputs)
            #filtered_duplicated_inputs = {k: torch.cat([v, v], -1) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
            #old_duplicated_outputs = model(**filtered_duplicated_inputs)
            #old_outputs = {k: v[..., int(v.shape[-2] / 2):, :] for k, v in old_duplicated_outputs.items()}

        model.set_adapter("<default>")
        outputs = model(**filtered_inputs)

        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        loss, metrics = self.dqn_loss(outputs, old_outputs, inputs, num_items_in_batch=num_items_in_batch)

        caller_function = inspect.stack()[1].function
        if caller_function == "training_step":
            train_eval = "train"
            prefix = ""
        elif caller_function == "prediction_step":
            train_eval = "eval"
            prefix = "eval_"
        else:
            raise ValueError(f"Invalid calling function {caller_function}")

        metrics = {f"{prefix}{k}": v for k, v in metrics.items()}
        self.store_metrics(metrics, train_eval=train_eval)

        if self.args.average_tokens_across_devices and self.model_accepts_loss_kwargs:
            loss *= self.accelerator.num_processes

        return (loss, outputs) if return_outputs else loss

    def store_metrics(self, metrics, train_eval = "train"):
        for key, value in metrics.items():
            self._stored_metrics[train_eval][key].append(value)

    def log(self, logs, start_time = None):
        # logs either has 'loss' or 'eval_loss'
        train_eval = "train" if "loss" in logs else "eval"
        # Add averaged stored metrics to logs
        for key, metrics in self._stored_metrics[train_eval].items():
            logs[key] = torch.tensor(metrics).mean().item()
        del self._stored_metrics[train_eval]

        return super().log(logs, start_time)

class polyak_update(TrainerCallback):
    def on_optimizer_step(self, *args, **kwargs):
        model = kwargs['model']
        source = get_adapter_state_dict(model, "<default>")
        target = get_adapter_state_dict(model, "<delayed>")
        polyak_dict(source, target, polyak_const)

In [14]:
from transformers import TrainingArguments
import transformers

learning_rate = 1e-2
lr_scheduler_type = "constant_with_warmup"
warmup_steps = 10
train_batch_size = 5
gradient_accumulation_steps = 2
num_train_epochs = 1
weight_decay = 0.01
eval_steps = 10
eval_batch_size = 5
eval_accumulation_steps = 2
run_eval = True
save_steps = eval_steps

#transformers.utils.logging.set_verbosity_debug()

training_config = TrainingArguments(
    output_dir=f"./trainer/{name}/",
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_steps=warmup_steps,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    eval_accumulation_steps=eval_accumulation_steps,
    num_train_epochs=num_train_epochs,
    fp16 = False,
    bf16 = True,
    fp16_full_eval = False,
    bf16_full_eval = True,
    optim="adamw_torch",
    weight_decay=weight_decay,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=save_steps,
    save_only_model=True,
    eval_strategy="steps" if run_eval else "no",
    eval_steps=eval_steps,
    disable_tqdm=True,
    report_to="wandb",
    overwrite_output_dir=True,
    run_name=name,
    eval_on_start=run_eval,
    #neftune_noise_alpha=15.0,
    push_to_hub=True,
    hub_always_push=True,
    hub_model_id=hf_name,
    hub_private_repo=True,
    hub_strategy="all_checkpoints",
    remove_unused_columns=False,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={
        "use_reentrant": False,
    },
)

trainer = CustomTrainer(
    model=model,
    args=training_config,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    callbacks=[polyak_update],
)

In [15]:
model.push_to_hub(hf_name, private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/micahr234/FrozenLake-mild-offset/commit/e6a25c2f100a7cba7145a9183147808e18054d2d', commit_message='Upload model', commit_description='', oid='e6a25c2f100a7cba7145a9183147808e18054d2d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/micahr234/FrozenLake-mild-offset', endpoint='https://huggingface.co', repo_type='model', repo_id='micahr234/FrozenLake-mild-offset'), pr_revision=None, pr_num=None)

In [16]:
wandb.run.log_code(".", name=name, include_fn=lambda path: path.endswith(".ipynb"))

<Artifact mild-offset>

In [ ]:
trainer.train()

{'eval_loss': 0.055589497089385986, 'eval_runtime': 13.497, 'eval_samples_per_second': 3.186, 'eval_steps_per_second': 0.667, 'eval_normalized_value_loss': 0.05584282428026199, 'eval_normalized_evalue_loss': 0.05584282428026199, 'eval_current_value': 23.543195724487305, 'eval_target_value': 22.11829376220703, 'eval_imitation_loss': 1.2178291082382202, 'eval_observation_loss': 0.7055228352546692, 'epoch': 0}
{'loss': 0.1162, 'grad_norm': 0.13237832486629486, 'learning_rate': 0.001, 'normalized_value_loss': 0.05810130760073662, 'normalized_evalue_loss': 0.05810130760073662, 'current_value': 23.52129554748535, 'target_value': 22.02776336669922, 'imitation_loss': 1.2346022129058838, 'observation_loss': 0.7029266357421875, 'epoch': 0.0010460251046025104}
{'loss': 0.0812, 'grad_norm': 0.0943995863199234, 'learning_rate': 0.002, 'normalized_value_loss': 0.04058469831943512, 'normalized_evalue_loss': 0.04058469831943512, 'current_value': 23.393001556396484, 'target_value': 22.3326358795166, 'i

No files have been modified since last commit. Skipping to prevent empty commit.


{'loss': 0.1053, 'grad_norm': 0.012028385885059834, 'learning_rate': 0.01, 'normalized_value_loss': 0.05266265571117401, 'normalized_evalue_loss': 0.05266265571117401, 'current_value': 21.36142349243164, 'target_value': 21.23766326904297, 'imitation_loss': 1.475055456161499, 'observation_loss': 2.691695213317871, 'epoch': 0.021966527196652718}
{'loss': 0.1127, 'grad_norm': 0.015352172777056694, 'learning_rate': 0.01, 'normalized_value_loss': 0.056367259472608566, 'normalized_evalue_loss': 0.056367259472608566, 'current_value': 20.92142677307129, 'target_value': 20.646535873413086, 'imitation_loss': 1.5570340156555176, 'observation_loss': 2.7372360229492188, 'epoch': 0.02301255230125523}
{'loss': 0.1033, 'grad_norm': 0.018701588734984398, 'learning_rate': 0.01, 'normalized_value_loss': 0.05167369544506073, 'normalized_evalue_loss': 0.05167369544506073, 'current_value': 21.116928100585938, 'target_value': 21.360029220581055, 'imitation_loss': 1.4530045986175537, 'observation_loss': 2.752

No files have been modified since last commit. Skipping to prevent empty commit.


{'loss': 0.0971, 'grad_norm': 0.02057694084942341, 'learning_rate': 0.01, 'normalized_value_loss': 0.04853811115026474, 'normalized_evalue_loss': 0.04853811115026474, 'current_value': 20.884769439697266, 'target_value': 21.179092407226562, 'imitation_loss': 1.4279603958129883, 'observation_loss': 2.6623988151550293, 'epoch': 0.032426778242677826}
{'loss': 0.0921, 'grad_norm': 0.025170082226395607, 'learning_rate': 0.01, 'normalized_value_loss': 0.04606999084353447, 'normalized_evalue_loss': 0.04606999084353447, 'current_value': 21.397144317626953, 'target_value': 20.952194213867188, 'imitation_loss': 1.4798880815505981, 'observation_loss': 2.438171148300171, 'epoch': 0.03347280334728033}
{'loss': 0.0887, 'grad_norm': 0.03148356452584267, 'learning_rate': 0.01, 'normalized_value_loss': 0.044356103986501694, 'normalized_evalue_loss': 0.044356103986501694, 'current_value': 21.6719970703125, 'target_value': 21.15502166748047, 'imitation_loss': 1.4276065826416016, 'observation_loss': 2.4931

...

{'loss': 0.0148, 'grad_norm': 0.011819240637123585, 'learning_rate': 0.01, 'normalized_value_loss': 0.007386468350887299, 'normalized_evalue_loss': 0.007386468350887299, 'current_value': 22.59405517578125, 'target_value': 22.491331100463867, 'imitation_loss': 1.7656023502349854, 'observation_loss': 7.144095420837402, 'epoch': 0.9320083682008368}
{'loss': 0.024, 'grad_norm': 0.014842900447547436, 'learning_rate': 0.01, 'normalized_value_loss': 0.01200406439602375, 'normalized_evalue_loss': 0.01200406439602375, 'current_value': 21.196279525756836, 'target_value': 21.04131317138672, 'imitation_loss': 2.118631362915039, 'observation_loss': 7.141097068786621, 'epoch': 0.9330543933054394}
{'loss': 0.0228, 'grad_norm': 0.012646357528865337, 'learning_rate': 0.01, 'normalized_value_loss': 0.011377595365047455, 'normalized_evalue_loss': 0.011377595365047455, 'current_value': 21.382457733154297, 'target_value': 21.532264709472656, 'imitation_loss': 1.8781616687774658, 'observation_loss': 6.75119

No files have been modified since last commit. Skipping to prevent empty commit.


{'loss': 0.0193, 'grad_norm': 0.011157426983118057, 'learning_rate': 0.01, 'normalized_value_loss': 0.00965551845729351, 'normalized_evalue_loss': 0.00965551845729351, 'current_value': 22.379619598388672, 'target_value': 22.564136505126953, 'imitation_loss': 1.903613567352295, 'observation_loss': 6.337939262390137, 'epoch': 0.9424686192468619}
{'loss': 0.017, 'grad_norm': 0.009340833872556686, 'learning_rate': 0.01, 'normalized_value_loss': 0.008486684411764145, 'normalized_evalue_loss': 0.008486684411764145, 'current_value': 22.029895782470703, 'target_value': 22.08753204345703, 'imitation_loss': 1.854692816734314, 'observation_loss': 6.263360977172852, 'epoch': 0.9435146443514645}
{'loss': 0.0229, 'grad_norm': 0.010220915079116821, 'learning_rate': 0.01, 'normalized_value_loss': 0.011427521705627441, 'normalized_evalue_loss': 0.011427521705627441, 'current_value': 22.827150344848633, 'target_value': 22.860368728637695, 'imitation_loss': 1.842536211013794, 'observation_loss': 6.597440

No files have been modified since last commit. Skipping to prevent empty commit.


{'loss': 0.0227, 'grad_norm': 0.01424375455826521, 'learning_rate': 0.01, 'normalized_value_loss': 0.011341117322444916, 'normalized_evalue_loss': 0.011341117322444916, 'current_value': 21.42429542541504, 'target_value': 21.29715919494629, 'imitation_loss': 2.237278938293457, 'observation_loss': 6.745580673217773, 'epoch': 0.952928870292887}
{'loss': 0.0218, 'grad_norm': 0.01192664448171854, 'learning_rate': 0.01, 'normalized_value_loss': 0.010909289121627808, 'normalized_evalue_loss': 0.010909289121627808, 'current_value': 21.859928131103516, 'target_value': 21.903343200683594, 'imitation_loss': 1.929279088973999, 'observation_loss': 6.82247257232666, 'epoch': 0.9539748953974896}
{'loss': 0.0215, 'grad_norm': 0.01223144680261612, 'learning_rate': 0.01, 'normalized_value_loss': 0.010771457105875015, 'normalized_evalue_loss': 0.010771457105875015, 'current_value': 20.751827239990234, 'target_value': 20.949249267578125, 'imitation_loss': 2.27070951461792, 'observation_loss': 6.7963027954

In [ ]:
wandb.finish()